In [2]:
import requests
from requests_html import HTMLSession, HTML
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import time

In [ ]:
# what do we want?
    # dashboard
    # allows you to filter by industry
    # dashboard shows only top 5 of cap valuation
    # what do we wwanna ssee on dashboard? candlestick of top 5 stocks
    # opeing price of top 5 stocks

In [3]:
url1 = 'https://en.wikipedia.org/wiki/S%26P_500_Dividend_Aristocrats'
url2 = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [4]:
s = HTMLSession()
r = s.get(url2)

In [5]:
r.html

<HTML url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'>

In [6]:
table = r.html.find('table', first=True)
table

<Element 'table' class=('wikitable', 'sortable') id='constituents'>

In [7]:
rows = table.find('tr')
len(rows)

In [8]:
row = rows[-1]
row

In [11]:
for i in row.find('td'):
    print(i.text)


ZTS
Zoetis
reports
Health Care
Pharmaceuticals
Parsippany, New Jersey
2013-06-21
0001555280
1952


In [12]:
table_details = [[c.text.strip() for c in row.find('td')] for row in rows[1:]]
table_details[:5]

In [14]:
columns = [
    'Symbol', 'Security', 'SEC Filings', 
    'GICS Sector', 'GICS Sub-Industry', 
    'Headquarters Location', 'Date First Added',
    'CIK', 'Founded']

In [15]:
df = pd.DataFrame(table_details, columns=columns)
df

,Symbol,Security,SEC Filings,GICS Sector,GICS Sub-Industry,Headquartes Location,Date First Added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
...,...,...,...,...,...,...,...,...,...
497,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,1997
498,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969
499,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,1927
500,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,0000109380,1873


In [96]:
def ticker_history(tickers):
    data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = tickers,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "ytd",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # Whether to ignore timezone when aligning ticker data from 
        # different timezones. Default is True. False may be useful for 
        # minute/hourly data.
        ignore_tz = False,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

    return data

In [17]:
df.loc[:5, 'Symbol']

0     MMM
1     AOS
2     ABT
3    ABBV
4    ABMD
5     ACN
Name: Symbol, dtype: object

In [18]:
test = ticker_history(list(df.loc[:5, 'Symbol']))

[*********************100%***********************]  6 of 6 completed


In [19]:
test['MMM']

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-03 00:00:00-05:00,173.069715,173.817033,170.662723,172.506790,1930700
2022-01-04 00:00:00-05:00,173.224972,175.923119,172.788225,174.923447,2522200
2022-01-05 00:00:00-05:00,171.914746,176.427830,171.788569,174.205261,2952400
2022-01-06 00:00:00-05:00,175.554318,175.942535,172.312647,172.759109,2505400
2022-01-07 00:00:00-05:00,172.885282,175.175797,172.002072,174.651688,2800200
...,...,...,...,...,...
2022-10-24 00:00:00-04:00,117.529999,119.139999,117.190002,118.379997,3394100
2022-10-25 00:00:00-04:00,113.510002,118.599998,113.150002,118.500000,5014700
2022-10-26 00:00:00-04:00,119.589996,123.290001,119.589996,122.699997,4628000


In [20]:
# push one of those columns into yfinance - symbol
# exploe the data

In [25]:
test['MMM']["Open"]

Date
2022-01-03 00:00:00-05:00    173.069715
2022-01-04 00:00:00-05:00    173.224972
2022-01-05 00:00:00-05:00    171.914746
2022-01-06 00:00:00-05:00    175.554318
2022-01-07 00:00:00-05:00    172.885282
                                ...    
2022-10-24 00:00:00-04:00    117.529999
2022-10-25 00:00:00-04:00    113.510002
2022-10-26 00:00:00-04:00    119.589996
2022-10-27 00:00:00-04:00    123.489998
2022-10-28 00:00:00-04:00    122.980003
Name: Open, Length: 208, dtype: float64

In [26]:
# we need a dataframe
# dataframe columns: name, industry, market capital

In [27]:
mmm = yf.Ticker("MMM")
mmm.info.keys()

{'zip': '55144-1000',
 'sector': 'Industrials',
 'fullTimeEmployees': 95000,
 'longBusinessSummary': '3M Company operates as a diversified technology company worldwide. It operates through four segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. The Transportation and Electronics segment provides ceramic solutions; attachment tapes, films, sound, and temperature management for transportation vehicles; premium large format graphic films for advertising an

In [28]:
mmm.info['marketCap']

69977260032

In [29]:
empt = list()

for ticker in tqdm(df['Symbol'].tolist()):
    ticker_info = yf.Ticker(ticker)
    # i hindsight, i should have probably relied on the get method
    try:
        name = ticker_info.info['longName']
    except KeyError:
        name = ticker_info.info['shortName']
    except:
        name = None
    
    try:
        sector = ticker_info.info['sector']
    except KeyError:
        sector = None

    try:
        business_summary = ticker_info.info['longBusinessSummary']
    except KeyError:
        business_summary = None

    try:
        industry = ticker_info.info['industry']
    except KeyError:
        industry = None

    try:
        market_capitalization = ticker_info.info['marketCap']  
    except KeyError:
        market_capitalization = None


    ticker_details = {
        'ticker': ticker,
        'name': name,
        'sector': sector,
        'business_summary': business_summary,
        'industry': industry,
        'market_capitalization': market_capitalization,
    }

    empt.append(ticker_details)

    time.sleep(2)

100%|██████████| 502/502 [39:21<00:00,  4.71s/it]  


In [31]:
df2 = pd.DataFrame(empt)
df2
# should convert to csv

,ticker,name,sector,business_summary,industry,market_capitalization
0,MMM,3M Company,Industrials,3M Company operates as a diversified technolog...,Conglomerates,6.997726e+10
1,AOS,A. O. Smith Corporation,Industrials,A. O. Smith Corporation manufactures and marke...,Specialty Industrial Machinery,8.398882e+09
2,ABT,Abbott Laboratories,Healthcare,"Abbott Laboratories, together with its subsidi...",Medical Devices,1.742289e+11
3,ABBV,AbbVie Inc.,Healthcare,"AbbVie Inc. discovers, develops, manufactures,...",Drug Manufacturers—General,2.609892e+11
4,ABMD,"Abiomed, Inc.",Healthcare,"Abiomed, Inc. engages in the research, develop...",Medical Devices,1.173164e+10
...,...,...,...,...,...,...
497,YUM,"Yum! Brands, Inc.",Consumer Cyclical,"YUM! Brands, Inc., together with its subsidiar...",Restaurants,3.455452e+10
498,ZBRA,Zebra Technologies Corporation,Technology,"Zebra Technologies Corporation, together with ...",Communication Equipment,1.539115e+10
499,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,"Zimmer Biomet Holdings, Inc., together with it...",Medical Devices,2.370479e+10
500,ZION,"Zions Bancorporation, National Association",Financial Services,"Zions Bancorporation, National Association pro...",Banks—Regional,8.064103e+09


In [37]:
df2.head()

,ticker,name,sector,business_summary,industry,market_capitalization
0,MMM,3M Company,Industrials,3M Company operates as a diversified technolog...,Conglomerates,6.997726e+10
1,AOS,A. O. Smith Corporation,Industrials,A. O. Smith Corporation manufactures and marke...,Specialty Industrial Machinery,8.398882e+09
2,ABT,Abbott Laboratories,Healthcare,"Abbott Laboratories, together with its subsidi...",Medical Devices,1.742289e+11
3,ABBV,AbbVie Inc.,Healthcare,"AbbVie Inc. discovers, develops, manufactures,...",Drug Manufacturers—General,2.609892e+11
4,ABMD,"Abiomed, Inc.",Healthcare,"Abiomed, Inc. engages in the research, develop...",Medical Devices,1.173164e+10


In [34]:
df2.sector.unique()

array(['Industrials', 'Healthcare', 'Technology',
       'Communication Services', 'Consumer Defensive',
       'Consumer Cyclical', 'Utilities', 'Financial Services',
       'Basic Materials', 'Real Estate', 'Energy', None], dtype=object)

In [36]:
df['GICS Sector'].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Staples',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Real Estate', 'Energy'], dtype=object)

In [38]:
df.head()

,Symbol,Security,SEC Filings,GICS Sector,GICS Sub-Industry,Headquartes Location,Date First Added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981


In [88]:
df2.isna().sum()

ticker                    0
name                      0
sector                    2
business_summary          2
industry                  2
market_capitalization    10
dtype: int64

In [89]:
df2[df2.market_capitalization.isna()]

,ticker,name,sector,business_summary,industry,market_capitalization
58,BALL,Ball Corporation,Consumer Cyclical,Ball Corporation supplies aluminum packaging p...,Packaging & Containers,NaN
64,BRK.B,Berkshire Hathaway,None,None,None,NaN
80,BF.B,Brown–Forman,None,None,None,NaN
126,CEG,Constellation Energy Corporation,Utilities,Constellation Energy Corporation generates and...,Utilities—Renewable,NaN
169,ELV,Elevance Health Inc.,Healthcare,Elevance Health Inc. operates as a health bene...,Healthcare Plans,NaN
361,PARA,Paramount Global,Communication Services,Paramount Global operates as a media and enter...,Entertainment,NaN
425,SBUX,Starbucks Corporation,Consumer Cyclical,"Starbucks Corporation, together with its subsi...",Restaurants,NaN
474,V,Visa Inc.,Financial Services,Visa Inc. operates as a payments technology co...,Credit Services,NaN
480,WBD,"Warner Bros. Discovery, Inc.",Communication Services,"Warner Bros. Discovery, Inc., a media company,...",Entertainment,NaN
492,WTW,Willis Towers Watson Public Limited Company,Financial Services,Willis Towers Watson Public Limited Company op...,Insurance Brokers,NaN


In [80]:
df2[df2.name.isna()]

,ticker,name,sector,business_summary,industry,market_capitalization
17,ALB,None,Basic Materials,"Albemarle Corporation develops, manufactures, ...",Specialty Chemicals,3.281486e+10
64,BRK.B,None,None,None,None,NaN
80,BF.B,None,None,None,None,NaN
227,PEAK,None,Real Estate,"Healthpeak Properties, Inc. is a fully integra...",REIT—Healthcare Facilities,1.282584e+10
262,JKHY,None,Technology,"Jack Henry & Associates, Inc. provides technol...",Information Technology Services,1.470267e+10


In [87]:
df2.name = df2.name.fillna(df['Security'])

In [127]:
brk = yf.Ticker("BF-B")
brk.info.keys()
# the issue is the dot instead of dash
#might try an str.contains ., if just 2 then eplace with -

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'f

In [128]:
brk.info['longName']

'Brown-Forman Corporation'

In [ ]:
df2.isna().sum()

In [70]:
sector_group = df2.groupby('sector')
[x[-1] for x in sector_group['market_capitalization'].nlargest(5).index.tolist()]

[286,
 415,
 14,
 130,
 210,
 24,
 23,
 309,
 151,
 433,
 26,
 234,
 443,
 305,
 337,
 479,
 380,
 117,
 367,
 132,
 190,
 104,
 123,
 174,
 410,
 266,
 302,
 59,
 484,
 102,
 463,
 264,
 170,
 369,
 3,
 461,
 394,
 235,
 459,
 289,
 34,
 382,
 134,
 386,
 178,
 45,
 315,
 347,
 356,
 77,
 336,
 159,
 422,
 154,
 413]

In [57]:
indexes = [x[-1] for x in sector_group['market_capitalization'].nlargest(5).index.tolist()]
# we can use this indexes to get the tickers and market cap of top 5 per sector

In [58]:
df2.loc[indexes]

,ticker,name,sector,business_summary,industry,market_capitalization
286,LIN,Linde plc,Basic Materials,Linde plc operates as an industrial gas and en...,Specialty Chemicals,1.492039e+11
415,SHW,The Sherwin-Williams Company,Basic Materials,"The Sherwin-Williams Company develops, manufac...",Specialty Chemicals,5.931660e+10
14,APD,"Air Products and Chemicals, Inc.",Basic Materials,"Air Products and Chemicals, Inc. provides atmo...",Specialty Chemicals,5.643454e+10
130,CTVA,"Corteva, Inc.",Basic Materials,"Corteva, Inc. operates in the agriculture busi...",Agricultural Inputs,4.812257e+10
210,FCX,Freeport-McMoRan Inc.,Basic Materials,Freeport-McMoRan Inc. engages in the mining of...,Copper,4.728474e+10
24,GOOG,Alphabet Inc.,Communication Services,Alphabet Inc. provides various products and pl...,Internet Content & Information,1.248045e+12
23,GOOGL,Alphabet Inc.,Communication Services,Alphabet Inc. provides various products and pl...,Internet Content & Information,1.248044e+12
309,META,"Meta Platforms, Inc.",Communication Services,"Meta Platforms, Inc. develops products that en...",Internet Content & Information,2.630337e+11
151,DIS,The Walt Disney Company,Communication Services,"The Walt Disney Company, together with its sub...",Entertainment,1.925811e+11
433,TMUS,"T-Mobile US, Inc.",Communication Services,"T-Mobile US, Inc., together with its subsidiar...",Telecom Services,1.886065e+11


In [98]:
history_df = ticker_history(df2.loc[indexes, 'ticker'].tolist())

[*********************100%***********************]  55 of 55 completed


              Date                     
AAPL  Open    2022-01-03 00:00:00-05:00    1.770972e+02
              2022-01-04 00:00:00-05:00    1.818774e+02
              2022-01-05 00:00:00-05:00    1.788698e+02
              2022-01-06 00:00:00-05:00    1.719883e+02
              2022-01-07 00:00:00-05:00    1.721775e+02
                                               ...     
SO    Volume  2022-01-03 00:00:00-05:00    3.616600e+06
              2022-01-04 00:00:00-05:00    5.259900e+06
              2022-01-05 00:00:00-05:00    4.542900e+06
              2022-01-06 00:00:00-05:00    4.015300e+06
              2022-01-07 00:00:00-05:00    4.380300e+06
Length: 1375, dtype: float64

In [100]:
history_df.head()

AAPL                                      \
                                 Open        High         Low       Close   
Date                                                                        
2022-01-03 00:00:00-05:00  177.097152  182.126343  176.977651  181.259918   
2022-01-04 00:00:00-05:00  181.877375  182.186095  178.381831  178.959442   
2022-01-05 00:00:00-05:00  178.869833  179.427523  173.920313  174.199158   
2022-01-06 00:00:00-05:00  171.988296  174.577587  170.932666  171.291183   
2022-01-07 00:00:00-05:00  172.177529  173.422378  170.325193  171.460495   

                                             PEP                          \
                              Volume        Open        High         Low   
Date                                                                       
2022-01-03 00:00:00-05:00  104487900  168.759010  169.729232  167.161591   
2022-01-04 00:00:00-05:00   99310400  169.209830  170.964055  168.759016   
2022-01-05 00:00:00-05:00   94537600  169.895835  171.855870  169.768427   
2022-01-06 00:00:00-05:00   96904000  170.434838  172.091069  169.768418   
2022-01-07 00:00:00-05:00   86709100  170.611235  171.199252  169.160814   

                                                ...        ABBV              \
                                Close   Volume  ...        Open        High   
Date                                            ...                           
2022-01-03 00:00:00-05:00  169.523422  5488900  ...  130.332162  130.611281   
2022-01-04 00:00:00-05:00  169.768433  4717600  ...  130.255191  131.111816   
2022-01-05 00:00:00-05:00  170.346649  6564600  ...  129.937547  132.969417   
2022-01-06 00:00:00-05:00  170.385834  4288700  ...  131.285029  131.535288   
2022-01-07 00:00:00-05:00  170.601440  4916000  ...  130.178188  130.746059   

                                                                   SO  \
                                  Low       Close   Volume       Open   
Date                                                                    
2022-01-03 00:00:00-05:00  128.503403  130.341782  6839800  66.593400   
2022-01-04 00:00:00-05:00  129.340819  130.091568  6298300  66.126757   
2022-01-05 00:00:00-05:00  129.937547  130.774918  7724900  65.767074   
2022-01-06 00:00:00-05:00  130.091538  130.158905  4667000  66.428133   
2022-01-07 00:00:00-05:00  129.100192  129.822067  8630300  65.990666   

                                                                     
                                High        Low      Close   Volume  
Date                                                                 
2022-01-03 00:00:00-05:00  66.632287  65.485131  66.272583  3616600  
2022-01-04 00:00:00-05:00  67.196146  66.039265  66.272583  5259900  
2022-01-05 00:00:00-05:00  67.050333  65.767074  66.350372  4542900  
2022-01-06 00:00:00-05:00  66.787830  65.922603  66.272583  4015300  
2022-01-07 00:00:00-05:00  67.215598  65.592082  66.729515  4380300  

[5 rows x 275 columns]

In [ ]:
# we want top 5 stocks by market cap per sector - done
# we want the ticker where the top 5 largest caps are
# we still need the history of those ticjkers that are top 5 - done

In [ ]:
# we have the indexes of the copanies that are in top 5
# but we also want to know how to do the where sql - different

In [90]:
df2_sorted = df2.sort_values(
    by='market_capitalization',
    ascending=False
)
df2_sorted.head(10)

,ticker,name,sector,business_summary,industry,market_capitalization
45,AAPL,Apple Inc.,Technology,"Apple Inc. designs, manufactures, and markets ...",Consumer Electronics,2.502867e+12
315,MSFT,Microsoft Corporation,Technology,"Microsoft Corporation develops, licenses, and ...",Software—Infrastructure,1.758286e+12
24,GOOG,Alphabet Inc.,Communication Services,Alphabet Inc. provides various products and pl...,Internet Content & Information,1.248045e+12
23,GOOGL,Alphabet Inc.,Communication Services,Alphabet Inc. provides various products and pl...,Internet Content & Information,1.248044e+12
26,AMZN,"Amazon.com, Inc.",Consumer Cyclical,"Amazon.com, Inc. engages in the retail sale of...",Internet Retail,1.053500e+12
463,UNH,UnitedHealth Group Incorporated,Healthcare,UnitedHealth Group Incorporated operates as a ...,Healthcare Plans,5.191859e+11
190,XOM,Exxon Mobil Corporation,Energy,Exxon Mobil Corporation explores for and produ...,Oil & Gas Integrated,4.716573e+11
264,JNJ,Johnson & Johnson,Healthcare,"Johnson & Johnson, together with its subsidiar...",Drug Manufacturers—General,4.571941e+11
479,WMT,Walmart Inc.,Consumer Defensive,Walmart Inc. engages in the operation of retai...,Discount Stores,3.953056e+11
266,JPM,JPMorgan Chase & Co.,Financial Services,JPMorgan Chase & Co. operates as a financial s...,Banks—Diversified,3.698178e+11


In [22]:
fig = px.line(test['MMM'], x=test['MMM'].index, y="Open")
fig.show()

In [23]:
fig = px.line(test['AOS'], x=test['AOS'].index, y="Open")
fig.show()

In [24]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    name="AOS",
    mode='lines',
    x=test['AOS'].index ,
    y=test['AOS']["Open"]
))

fig.add_trace(go.Scatter(
    name="MMM",
    mode='lines',
    x=test['MMM'].index ,
    y=test['MMM']["Open"]
))

fig.show()

In [55]:
candlestick = go.Candlestick(
                            x=test['MMM'].index,
                            open=test['MMM']["Open"],
                            high=test['MMM']['High'],
                            low=test['MMM']['Low'],
                            close=test['MMM']['Close']
                            )

fig = go.Figure(data=[candlestick])

fig.update_layout(
    title="MMM, Feb - 2022",
    yaxis_title='MMM Stock'
)

fig.show()